In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import glob
file_name_Despress = glob.glob('/content/drive/MyDrive/Colab Notebooks/Khanh/Depresjon/condition/*.csv')
file_name_Normal = glob.glob('/content/drive/MyDrive/Colab Notebooks/Khanh/Depresjon/control/*.csv')

In [5]:
def readfile(file_name):
    ID = 0
    result = []
    for f in file_name:
        df = pd.read_csv(f)
        out = [x[11:] for x in df.timestamp]
        
        df.timestamp = out     
        df_Pivot = pd.pivot_table(df,index=["timestamp"],columns=["date"],fill_value=0)
        if (df_Pivot.shape[-1]>7):
            #print(df_Pivot.shape[-1])
            for j in range(7, df_Pivot.shape[-1]+1):
                result.append(df_Pivot.iloc[:, j-7:j])
                #print(df_Pivot.iloc[:, j-7:j].shape)
        #print(np.array(result).shape)
    return np.array(result)
        

In [6]:
df_Depress = readfile(file_name_Despress)
df_Nomal = readfile(file_name_Normal)

In [7]:
print(df_Depress.shape)
print(df_Nomal.shape)

(267, 1440, 7)
(547, 1440, 7)


In [8]:
df_Final = np.concatenate((df_Depress, df_Nomal))

In [9]:
label =[]
for i in range(0, df_Depress.shape[0]+df_Nomal.shape[0]):
    if i <df_Depress.shape[0]:
        label.append(1)
    else:
        label.append(0)

In [10]:
df_Final.shape

(814, 1440, 7)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_Final, label, test_size=0.2, random_state=42)

In [12]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*7)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*7)

In [13]:
from sklearn.metrics import confusion_matrix
    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()

    
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    
    
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

In [14]:
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical

# one hot encode
y_train = to_categorical(y_train)
# one hot encode
y_test = to_categorical(y_test)

In [15]:
from tensorflow.keras.callbacks import LearningRateScheduler

# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    if epoch % 50 == 0 and epoch:
        lr = lr - lr*0.1
        return lr
    return lr

In [22]:
# fit model on dataset
from tensorflow.keras.layers import GRU, LSTM
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense
def fit_model(X, y, filename, epoch):
    #X = X.reshape((X.shape[0], X.shape[1], 1))
    
    print(X.shape)
    print(y.shape)
    time_steps = X.shape[1]
    input_dim = 1
    print((time_steps, input_dim))
    
    #model_path = 'DL_model_Bagging/model_'+str(iteration)+'.h5'
    # define the keras model
    model_input = Input(shape=(time_steps, input_dim))
    x = GRU(128, return_sequences=True)(model_input)
    x = Attention(64)(x)
    x = Dense(32, activation = "relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(2, activation = "sigmoid")(x)
    model = Model(model_input, x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #print(model.summary())
    
    # train.
    model.fit(X, y, epochs=epoch, batch_size=190, validation_split=0.2,verbose=1, 
                           callbacks = [LearningRateScheduler(lr_scheduler, verbose=0),
                                        tensorflow.keras.callbacks.ModelCheckpoint(filename,
                                                                                   monitor='val_loss',
                                                                                   save_best_only=True,
                                                                                   mode='min',
                                                                                   verbose=0)])
    #print("======================= Done for iteration ", i) 

Making prediction

In [17]:
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = '/content/drive/MyDrive/Colab Notebooks/Khanh/models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

In [18]:
pip install attention

In [19]:
# stacked generalization with neural net meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from numpy import argmax
import tensorflow
# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(248, activation='relu')(merge)
    hidden = Dense(64, activation='relu')(hidden)
    hidden = Dense(16, activation='relu')(hidden)
    output = Dense(2, activation='sigmoid')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
    #plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'],
                 )
    
    #model.summary()
    return model

# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
    # prepare input data
    #filename = 'models/final.h5'
    X = [inputX for _ in range(len(model.input))]
    # encode output data
    inputy_enc = inputy #to_categorical(inputy)
    print(inputy_enc.shape)
    # fit model
    model.fit(X, inputy_enc, epochs=100, verbose=1, batch_size=120, validation_split=0.2,
              callbacks = [LearningRateScheduler(lr_scheduler, verbose=0),
                                    tensorflow.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Khanh/models/Ensebmle.h5',
                                                                               monitor='val_loss',
                                                                               save_best_only=True,
                                                                               mode='min',
                                                                             verbose=0)])
    model = load_model('/content/drive/MyDrive/Colab Notebooks/Khanh/models/Ensebmle.h5') 
    
    
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]
    # make prediction
    return model.predict(X)

# generate 2d classification dataset

print(X_train.shape, X_test.shape)
# load all models

#members = load_all_models(n_members)
#print('Loaded %d models' % len(members))

(651, 10080) (163, 10080)


In [20]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

Making **prediction**

In [23]:
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
from attention import Attention
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
from tqdm import tqdm
import warnings
import timeit
import tracemalloc
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from pathlib import Path
from tensorflow.keras import layers
    
warnings.filterwarnings('ignore')
# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve


max_auc = 0.93
for num_Interation in range(10, 50, 5):
    rsFile = "/content/drive/MyDrive/Colab Notebooks/Khanh/Result_OverIteration_Bagging_Numeric/NumberOfBag_"+str(num_Interation)+".csv"
    '''if (Path(rsFile).is_file()):
        continue
    else:
        '''
    f2=open(rsFile,"w")
    f2.write('num_Interation, epoch, Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for epoch in (100, 500):
        print("working on num_Interation: " + str(num_Interation) + "_with_epoch_"+str(epoch))
        
        
        
        # ============== Starting training
        start = timeit.default_timer()
        tracemalloc.start()

        ## Start training
        for i in range(0, num_Interation):
            # save model
            filename = 'models/model_' + str(i + 1) + '.h5'
            # fit model
            model = fit_model(X_train, y_train, filename, epoch)
            #model = fit_model(trainX, trainy)
            #model.save(filename)
            print('>Saved %s' % filename)
        current, peak = tracemalloc.get_traced_memory()
        stop = timeit.default_timer()
        Time_train=stop - start
        Train_Peak_RAM = peak / 10**6
        #print('traning time: ', Time_train)
        #print('Peak_Time:', Train_Peak_RAM)


        # ============== Starting predicting
        members = load_all_models(num_Interation)
        # define ensemble model
        stacked_model = define_stacked_model(members)
        #stacked_model.name = 'stacked_model_iter_'+str(num_Interation)+'_Epoc_'+str(epoch)
        # fit stacked model on test dataset
        fit_stacked_model(stacked_model, X_train, y_train)


        start = timeit.default_timer()
        tracemalloc.start()

        predicted = predict_stacked_model(stacked_model, X_test)

        stop = timeit.default_timer()
        Time_predict=stop - start
        current, peak = tracemalloc.get_traced_memory()
        Peak_Time_prediction = peak / 10**6


        auc=roc_auc_score(np.argmax(y_test, axis=1), predicted[:, 1])
        print('y_test.shape ' ,y_test.shape)
        print('==========================================')


        if max_auc <= auc:
            #max_auc = auc
            print('===================: ')
            print('======= AUC ==============: ', auc)
            print('=====AUC===AUC=====: ')
            print('===AUC========AUC==: ')
            print('=AUC===========AUC=: ')
            print('=AUC===========AUC=: ')
            print('=AUC===========AUC=: ')
            print('===================: ')
            print('===================: ')



        thres = predicted[:, 1].min()
        while (thres <= predicted[:, 1].max()):
            thres = thres+0.002
            pred = []
            for i in range(len(predicted)):
                if predicted[i][1]>thres:
                    pred.append(1)
                else:
                    pred.append(0)

            pred = np.array(pred)      

            tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(np.argmax(y_test, axis=1), pred)
            #auc = roc_plot(y_test, yhat)
            f2.write(str(num_Interation)+", "+str(epoch)+", "+str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")

        del(stacked_model)
    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       
        

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Epoch 151/500
3/3 [==============================] - 2s 789ms/step - loss: 0.0239 - accuracy: 0.9962 - val_loss: 0.6552 - val_accuracy: 0.9313
Epoch 152/500
3/3 [==============================] - 2s 795ms/step - loss: 0.0206 - accuracy: 0.9942 - val_loss: 0.6572 - val_accuracy: 0.9313
Epoch 153/500
3/3 [==============================] - 2s 808ms/step - loss: 0.0241 - accuracy: 0.9942 - val_loss: 0.6559 - val_accuracy: 0.9313
Epoch 154/500
3/3 [==============================] - 2s 786ms/step - loss: 0.0238 - accuracy: 0.9942 - val_loss: 0.6537 - val_accuracy: 0.9313
Epoch 155/500
3/3 [==============================] - 2s 801ms/step - loss: 0.0237 - accuracy: 0.9942 - val_loss: 0.6585 - val_accuracy: 0.9389
Epoch 156/500
3/3 [==============================] - 2s 794ms/step - loss: 0.0195 - accuracy: 0.9962 - val_loss: 0.6636 - val_accuracy: 0.9389
Epoch 157/500
3/3 [==============================] - 2s 805ms/step - loss: 0.0200 - a

InternalError: ignored